# Chapter 11: Development Tools and Frameworks

---

Building smart contracts is just the beginning. To develop robust, secure, and maintainable decentralized applications, you need a professional development environment. This chapter explores the most popular tools and frameworks used by blockchain developers, from Hardhat and Foundry to Remix and package managers. By the end, you'll know how to set up a complete development pipeline for testing, deploying, and managing smart contracts.

---

## 11.1 Hardhat Framework

Hardhat is a development environment for Ethereum software. It helps you compile, deploy, test, and debug your smart contracts. Think of it as a Swiss Army knife for smart contract development – it handles everything from compilation to testing to deployment, with an extensible plugin system.

### 11.1.1 Installation and Setup

**Prerequisites:**
- Node.js (v16+)
- npm or yarn

```bash
# Create a new project directory
mkdir my-hardhat-project
cd my-hardhat-project

# Initialize npm project
npm init -y

# Install Hardhat
npm install --save-dev hardhat

# Initialize Hardhat project
npx hardhat init
```

During initialization, you'll be prompted:
```
? What do you want to do? 
  Create a JavaScript project
  Create a TypeScript project
  Create an empty hardhat.config.js
  Quit
```

Choose "Create a JavaScript project" (or TypeScript if preferred). It will create a sample project with a contract, tests, and deployment script.

**Install additional recommended packages:**
```bash
npm install --save-dev @nomicfoundation/hardhat-toolbox
npm install --save-dev @openzeppelin/contracts
npm install --save-dev dotenv
```

The `hardhat-toolbox` includes many useful plugins (Etherscan verification, gas reporter, etc.).

### 11.1.2 Project Structure

After initialization, your project will look like this:

```
my-hardhat-project/
├── contracts/           # Solidity contracts
│   └── Lock.sol         # Sample contract
├── scripts/             # Deployment and utility scripts
│   └── deploy.js        # Sample deployment script
├── test/                # Test files
│   └── Lock.js          # Sample tests
├── hardhat.config.js    # Hardhat configuration
├── package.json         # Dependencies
└── .gitignore           # Git ignore file
```

**`hardhat.config.js` example:**
```javascript
require("@nomicfoundation/hardhat-toolbox");
require("dotenv").config();

module.exports = {
  solidity: "0.8.19",
  networks: {
    sepolia: {
      url: process.env.SEPOLIA_RPC_URL || "",
      accounts: process.env.PRIVATE_KEY ? [process.env.PRIVATE_KEY] : [],
    },
  },
  etherscan: {
    apiKey: process.env.ETHERSCAN_API_KEY,
  },
  gasReporter: {
    enabled: true,
    currency: "USD",
  },
};
```

### 11.1.3 Writing Tests

Hardhat uses Mocha and Chai for testing. Tests are written in JavaScript (or TypeScript) and can be run with `npx hardhat test`.

**Example test (`test/Token.js`):**
```javascript
const { expect } = require("chai");
const { ethers } = require("hardhat");

describe("Token contract", function () {
  let Token, token, owner, addr1, addr2;

  beforeEach(async function () {
    // Get the contract factory and signers
    Token = await ethers.getContractFactory("MyToken");
    [owner, addr1, addr2] = await ethers.getSigners();

    // Deploy the contract
    token = await Token.deploy(1000); // initial supply
  });

  describe("Deployment", function () {
    it("Should set the right owner", async function () {
      expect(await token.owner()).to.equal(owner.address);
    });

    it("Should assign the total supply to the owner", async function () {
      const ownerBalance = await token.balanceOf(owner.address);
      expect(await token.totalSupply()).to.equal(ownerBalance);
    });
  });

  describe("Transactions", function () {
    it("Should transfer tokens between accounts", async function () {
      // Transfer 50 tokens from owner to addr1
      await token.transfer(addr1.address, 50);
      expect(await token.balanceOf(addr1.address)).to.equal(50);

      // Transfer 50 tokens from addr1 to addr2
      await token.connect(addr1).transfer(addr2.address, 50);
      expect(await token.balanceOf(addr2.address)).to.equal(50);
    });

    it("Should fail if sender doesn't have enough tokens", async function () {
      const initialOwnerBalance = await token.balanceOf(owner.address);
      
      // Try to send more tokens than owner has
      await expect(
        token.connect(addr1).transfer(owner.address, 1)
      ).to.be.revertedWith("ERC20: insufficient balance");

      // Owner balance should remain unchanged
      expect(await token.balanceOf(owner.address)).to.equal(initialOwnerBalance);
    });
  });
});
```

**Running tests:**
```bash
npx hardhat test
```

### 11.1.4 Deployment Scripts

Deployment scripts are JavaScript files that run on the Hardhat network. They use the `ethers` library to interact with the blockchain.

**Example deployment script (`scripts/deploy.js`):**
```javascript
const { ethers } = require("hardhat");

async function main() {
  console.log("Deploying MyToken...");

  // Get the contract factory
  const MyToken = await ethers.getContractFactory("MyToken");
  
  // Deploy with initial supply of 1,000,000 tokens
  const initialSupply = ethers.parseUnits("1000000", 18);
  const token = await MyToken.deploy(initialSupply);

  await token.waitForDeployment();

  console.log("MyToken deployed to:", await token.getAddress());
}

main()
  .then(() => process.exit(0))
  .catch((error) => {
    console.error(error);
    process.exit(1);
  });
```

**Deploy to a local network:**
```bash
npx hardhat run scripts/deploy.js
```

**Deploy to a testnet (e.g., Sepolia):**
```bash
npx hardhat run scripts/deploy.js --network sepolia
```

### 11.1.5 Hardhat Network and Forking

Hardhat comes with a built-in local Ethereum network (Hardhat Network) that's perfect for testing. One of its most powerful features is **forking** – you can simulate the state of a live network (like Ethereum mainnet) locally.

**Forking mainnet in `hardhat.config.js`:**
```javascript
module.exports = {
  networks: {
    hardhat: {
      forking: {
        url: "https://eth-mainnet.g.alchemy.com/v2/YOUR_API_KEY",
        blockNumber: 15000000 // optional: specific block
      }
    }
  }
};
```

Now when you run tests or scripts, you have access to all mainnet contracts and balances. This is invaluable for testing interactions with existing protocols.

### 11.1.6 Plugins and Extensions

Hardhat's plugin system extends its functionality. The `hardhat-toolbox` includes several essential plugins:

- `@nomicfoundation/hardhat-verify`: Verify contracts on Etherscan.
- `hardhat-gas-reporter`: Estimate gas costs.
- `solidity-coverage`: Measure test coverage.
- `@nomicfoundation/hardhat-chai-matchers`: Enhanced Chai matchers.

**Installing a custom plugin:**
```bash
npm install --save-dev @openzeppelin/hardhat-upgrades
```

Then require it in `hardhat.config.js`:
```javascript
require("@openzeppelin/hardhat-upgrades");
```

**Popular plugins:**
- `hardhat-deploy`: Manage deployments with tags and migrations.
- `hardhat-contract-sizer`: Check contract sizes.
- `hardhat-abi-exporter`: Export ABIs to a folder.

---

## 11.2 Foundry Framework

Foundry is a fast, portable, and modular toolkit for Ethereum application development written in Rust. It's becoming increasingly popular because of its speed and the ability to write tests in Solidity directly.

### 11.2.1 Installation (Forge, Cast, Anvil)

Foundry consists of:
- **Forge**: Compiles, tests, and deploys contracts.
- **Cast**: A CLI for interacting with contracts (sending transactions, querying data).
- **Anvil**: A local testnet node (like Ganache, but faster).

**Installation (macOS/Linux):**
```bash
curl -L https://foundry.paradigm.xyz | bash
foundryup
```

For Windows, use WSL or follow instructions on the [Foundry book](https://book.getfoundry.sh/).

### 11.2.2 Project Setup

```bash
# Create a new project
forge init my-foundry-project
cd my-foundry-project

# Build the project
forge build
```

**Project structure:**
```
my-foundry-project/
├── src/                 # Contracts
│   └── Counter.sol
├── test/                # Solidity tests
│   └── Counter.t.sol
├── script/              # Deployment scripts
│   └── Counter.s.sol
├── lib/                 # Dependencies (git submodules)
│   └── forge-std/
├── foundry.toml         # Configuration
└── remappings.txt       # Import remappings
```

### 11.2.3 Writing Tests in Solidity

Foundry allows you to write tests in Solidity using the `forge-std` library. This is a huge advantage because you can test your contracts in the same language they're written in, and leverage advanced testing techniques like fuzzing and mocking.

**Example test (`test/Counter.t.sol`):**
```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.19;

import "forge-std/Test.sol";
import "../src/Counter.sol";

contract CounterTest is Test {
    Counter public counter;

    function setUp() public {
        counter = new Counter();
        counter.setNumber(0);
    }

    function testIncrement() public {
        counter.increment();
        assertEq(counter.number(), 1);
    }

    function testSetNumber(uint256 x) public {
        // Fuzzing: test with random inputs
        counter.setNumber(x);
        assertEq(counter.number(), x);
    }

    function testFailIncrementZero() public {
        // Expect revert when incrementing beyond max
        vm.assume(counter.number() == type(uint256).max);
        counter.increment();
    }
}
```

**Running tests:**
```bash
forge test
```

**With gas reports:**
```bash
forge test --gas-report
```

### 11.2.4 Fast Compilation and Testing

Foundry is incredibly fast because it's written in Rust and compiles contracts using the Solidity compiler directly, without JavaScript overhead. It also supports:

- **Fuzzing**: Automatic testing with random inputs.
- **Cheatcodes**: Special functions (like `vm.prank`, `vm.warp`) to manipulate the EVM state during tests.

**Common cheatcodes:**
```solidity
// Set msg.sender for the next call
vm.prank(address(0x123));
counter.increment();

// Set block.timestamp
vm.warp(block.timestamp + 1 days);

// Set block number
vm.roll(block.number + 100);

// Deal tokens to an address
deal(address(this), 100 ether);

// Expect a revert
vm.expectRevert("Not authorized");
counter.withdraw();
```

### 11.2.5 Deployment with Forge

Deployment scripts are written in Solidity, using the `forge-std/Script.sol` library.

**Example deployment script (`script/Counter.s.sol`):**
```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.19;

import "forge-std/Script.sol";
import "../src/Counter.sol";

contract CounterScript is Script {
    function run() external {
        uint256 deployerPrivateKey = vm.envUint("PRIVATE_KEY");
        vm.startBroadcast(deployerPrivateKey);

        Counter counter = new Counter();

        vm.stopBroadcast();
    }
}
```

**Run the script:**
```bash
# Local Anvil node (must be running in another terminal)
forge script script/Counter.s.sol --broadcast

# Deploy to testnet
forge script script/Counter.s.sol --rpc-url sepolia --broadcast --verify
```

---

## 11.3 Truffle Suite (Legacy)

Truffle was the first popular Ethereum development framework. While newer tools have emerged, many legacy projects still use Truffle.

### 11.3.1 Overview and Setup

Truffle provides a development environment, testing framework, and asset pipeline.

**Installation:**
```bash
npm install -g truffle
```

**Create a project:**
```bash
truffle init my-truffle-project
cd my-truffle-project
```

**Project structure:**
```
my-truffle-project/
├── contracts/          # Solidity contracts
├── migrations/         # Deployment scripts (JavaScript)
├── test/               # Tests (JavaScript or Solidity)
├── truffle-config.js   # Configuration
└── package.json
```

### 11.3.2 Migration Scripts

Truffle uses "migrations" to deploy contracts. Each migration is a numbered JavaScript file.

**Example migration (`migrations/2_deploy_contracts.js`):**
```javascript
const MyToken = artifacts.require("MyToken");

module.exports = function (deployer) {
  deployer.deploy(MyToken, 1000000);
};
```

**Deploy:**
```bash
truffle migrate --network development
```

**Testing:**
```javascript
// test/mytoken.js
const MyToken = artifacts.require("MyToken");

contract("MyToken", (accounts) => {
  it("should put 1000000 tokens in the first account", async () => {
    const instance = await MyToken.deployed();
    const balance = await instance.balanceOf(accounts[0]);
    assert.equal(balance.toString(), "1000000");
  });
});
```

```bash
truffle test
```

While Truffle is still functional, Hardhat and Foundry have largely replaced it due to better performance, modern features, and active development.

---

## 11.4 Brownie Framework (Python)

Brownie is a Python-based development and testing framework for Ethereum smart contracts. It's popular among data scientists and Python developers who want to work in a familiar environment.

### 11.4.1 Setup and Usage

**Installation:**
```bash
pip install eth-brownie
```

**Create a project:**
```bash
brownie init my-brownie-project
cd my-brownie-project
```

**Project structure:**
```
my-brownie-project/
├── contracts/          # Solidity contracts
├── scripts/            # Python scripts
├── tests/              # Python tests
├── build/              # Compiled contracts
├── brownie-config.yaml # Configuration
└── package.json        # Dependencies
```

### 11.4.2 Python-Based Testing

Tests are written in Python using pytest. Brownie provides fixtures and helpers.

**Example test (`tests/test_token.py`):**
```python
from brownie import MyToken, accounts

def test_deployment():
    owner = accounts[0]
    token = MyToken.deploy(1000, {'from': owner})
    
    assert token.totalSupply() == 1000
    assert token.balanceOf(owner) == 1000

def test_transfer():
    owner = accounts[0]
    receiver = accounts[1]
    token = MyToken.deploy(1000, {'from': owner})
    
    token.transfer(receiver, 100, {'from': owner})
    
    assert token.balanceOf(owner) == 900
    assert token.balanceOf(receiver) == 100
```

**Run tests:**
```bash
brownie test
```

**Deploy:**
```bash
brownie run scripts/deploy.py
```

Brownie integrates well with Python data analysis tools, making it great for DeFi research and quantitative analysis.

---

## 11.5 Integrated Development Environments

IDEs enhance productivity with syntax highlighting, autocomplete, debugging, and integration with the tools above.

### 11.5.1 Remix IDE Advanced Features

Remix is not just for beginners; it offers advanced features for professional development:

- **Static analysis**: Built-in security analyzer.
- **Debugger**: Step through transactions, inspect memory and storage.
- **Plugin system**: Extend with Solidity unit testing, 3box, etc.
- **Workspaces**: Organize multiple projects.
- **File explorers**: Connect to GitHub, IPFS, or local filesystem.

**Using Remix with Hardhat:**
You can use the Remixd plugin to connect Remix to your local Hardhat project, combining the power of Remix's UI with Hardhat's testing/deployment.

### 11.5.2 VS Code Extensions for Solidity

Visual Studio Code is the most popular code editor for Solidity development. Essential extensions:

| Extension | Purpose |
|-----------|---------|
| **Solidity (by Nomic Foundation)** | Syntax highlighting, compilation, code completion, and integration with Hardhat/Foundry. |
| **Hardhat** | Task runner integration. |
| **Prettier - Code formatter** | Auto-format Solidity and JavaScript. |
| **ESLint** | Linting for JavaScript/TypeScript scripts. |
| **GitLens** | Git integration. |
| **Material Icon Theme** | Better file icons. |

**VS Code settings for Solidity (`.vscode/settings.json`):**
```json
{
  "editor.formatOnSave": true,
  "[solidity]": {
    "editor.defaultFormatter": "NomicFoundation.hardhat-solidity"
  },
  "solidity.compileUsingRemoteVersion": "latest",
  "solidity.packageDefaultDependenciesContractsDirectory": "contracts",
  "solidity.packageDefaultDependenciesDirectory": "node_modules"
}
```

### 11.5.3 Debugger Tools

Debugging transactions can be challenging. Tools to help:

- **Hardhat Console**: Run `npx hardhat console` for an interactive JavaScript environment.
- **Hardhat Tracer**: Plugin to trace transactions and view internal calls.
- **Tenderly**: Web-based debugger with visual transaction traces.
- **EVM.codes**: Reference for EVM opcodes.

**Example: Debugging in Hardhat**
```javascript
// In your test or script, use hardhat's built-in console.log
const { ethers } = require("hardhat");

// Add console.log to your Solidity contract:
import "hardhat/console.sol";

contract MyContract {
    function doSomething(uint x) public {
        console.log("doSomething called with x =", x);
        // ...
    }
}
```

Then run your test with:
```bash
npx hardhat test
```

---

## 11.6 Package Management

Managing dependencies is crucial for using established, audited code (like OpenZeppelin contracts) and reusing libraries.

### 11.6.1 npm in Blockchain Projects

npm is the default package manager for JavaScript-based projects (Hardhat, Truffle). It manages both JavaScript dependencies and Solidity contracts.

**Installing a Solidity package:**
```bash
npm install @openzeppelin/contracts
```

This installs the contracts in `node_modules/@openzeppelin/contracts`. You can then import them in your Solidity files:

```solidity
import "@openzeppelin/contracts/token/ERC20/ERC20.sol";
```

Hardhat and Truffle automatically resolve imports from `node_modules`. For Foundry, which uses git submodules, you'd use:

```bash
forge install OpenZeppelin/openzeppelin-contracts
```

### 11.6.2 OpenZeppelin Contracts Library

OpenZeppelin is the most trusted library for secure smart contract development. It provides:

- **Token standards**: ERC20, ERC721, ERC1155 implementations.
- **Access control**: Ownable, Roles, AccessControl.
- **Security**: ReentrancyGuard, Pausable.
- **Utilities**: SafeMath (for older Solidity), Address, Arrays.

**Example: Creating an ERC20 token with OpenZeppelin:**
```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.19;

import "@openzeppelin/contracts/token/ERC20/ERC20.sol";
import "@openzeppelin/contracts/access/Ownable.sol";

contract MyToken is ERC20, Ownable {
    constructor() ERC20("MyToken", "MTK") {
        _mint(msg.sender, 1000000 * 10 ** decimals());
    }

    function mint(address to, uint256 amount) public onlyOwner {
        _mint(to, amount);
    }
}
```

### 11.6.3 Using External Libraries

Beyond OpenZeppelin, many other useful libraries exist:

- **Uniswap V3 Core / Periphery**: For building DEX integrations.
- **Chainlink**: For oracles and randomness.
- **PRBMath**: Advanced fixed-point arithmetic.
- **Solmate**: Gas-optimized contracts by Rari Capital.

**Installing multiple dependencies:**
```bash
npm install @uniswap/v3-core @chainlink/contracts prb-math
```

**Managing versions:** Always lock versions in `package.json` to avoid unexpected changes.

---

## Chapter Summary

| Tool | Best For | Key Features |
|------|----------|--------------|
| **Hardhat** | Professional JavaScript/TypeScript developers | Extensible, great debugging, mainnet forking |
| **Foundry** | Speed enthusiasts, Solidity-native testing | Ultra-fast, Solidity tests, fuzzing, cheatcodes |
| **Truffle** | Legacy projects, simple setups | Migrations, mature ecosystem |
| **Brownie** | Python developers, data analysis | Pythonic, pytest integration |
| **Remix** | Quick prototyping, learning | Browser-based, no setup |
| **VS Code** | Daily development | Extensions, git integration, formatting |
| **OpenZeppelin** | Secure contract development | Audited implementations, standards |

**Choosing the right tool:**
- For most new projects, **Hardhat** is the safe, feature-rich choice.
- If you prioritize speed and Solidity-native testing, **Foundry** is excellent.
- For quick experiments, use **Remix**.
- If you're a Python developer, **Brownie** is a natural fit.

**Next Chapter Preview:** Chapter 12 – DApp Architecture. We'll explore how to build complete decentralized applications, combining smart contracts with frontend frameworks and decentralized storage.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='10. smart_contract_development.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='../4. decentralized_applications/12. dapp_architecture.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
